# Codice scraping da google news 

###Installazione librerie

In [ ]:
pip install requests_html

In [ ]:
pip install pygooglenews

## Funzione di scraping

In [ ]:
# Importiamo le librerie
import pandas as pd
from pygooglenews import GoogleNews                           
from datetime import datetime, timedelta
import urllib.request
from lxml.html import fromstring
import requests
import re

# Definiamo una funzione le cui variabili sono: search (parole chiave della ricerca), start_date (data inizio ricerca), end_date (data fine ricerca)
def get_storyII(search, start_date, end_date):
  # Definiamo story e df_stories come variabili globali
  global story
  global df_stories
  # Settiamo le impostazioni della ricerca: lingua inglese (lang), frequenza giornaliera (timedelta), insieme di date (data_list)
  gn = GoogleNews(lang = 'en')
  delta = timedelta(days = 1)  
  date_list = pd.date_range(start_date, end_date).tolist()  
  stories = []
  
  HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)'
                  ' AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'
             }
  # Per ogni giorno: cerca le notizie su google_news (result['entries'])
  for date in date_list[:-1]:   #helper = True--> ottimizza la ricerca delle parole chiave 
      result = gn.search(query = search, helper = True, from_=date.strftime('%Y-%m-%d'), to_=(date+delta).strftime('%Y-%m-%d'))   #seleziona pagina google news in un dato giorno
      newsitem = result['entries']   #riporta tutti gli oggetti della pagina google in un dato giorno (così per ogni giorno nel range)

      
      for item in newsitem:    # Apri ogni articolo e ricerca: titolo, link, data e twitter_link (se esistente)           
        try:
            # Verifica che venga aperto l'articolo
          print(item.link, item.published[4:16])   
          r = requests.get(item.link, headers=HEADERS, allow_redirects=False)    # apri la pagina dell'articolo
          tree = fromstring(r.content)                        # mappa la struttura della pagina HTML
          try:
            # trovi tutti i link nella pagina
            links = tree.xpath('//a/@href')                     
            f = re.compile('(https://twitter.com)/(\w+)/(status)/(\d+)')                
            # se trovi uno o più twitter_link, aggiungili ad una lista 
            twitter_links = [f.search(link).group() for link in links if f.search(link)]
            twitter_id = [f.search(link).group(4) for link in links if f.search(link)]
            story = {
                  'title':item.title,
                  'link':item.link,
                  'published':item.published[4:16],
                  'twitter_links': twitter_links,
                  'twitter_id': twitter_id  
            } 
            stories.append(story)
          except:
            continue
        except:
          continue  
      df_stories = pd.DataFrame(stories)
  return(df_stories)
      

In [ ]:
#@title
#prova
get_storyII("Iron Man", '2008-1-26','2008-6-2')
#"Iron Man": [ '2008-1-26', '2008-6-2'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

film_title = {'Phase 1': {"Iron Man": [ '2008-1-26', '2008-6-2'], 
                          "The Incredible Hulk": ['2008-02-01', '2008-07-13'],
                          "Iron Man 2": [ '2009-07-23', '2010-06-07'], 
                          "Thor": ['2010-12-09', '2011-06-06'],
                          "Captain America: The First Avenger": ['2011-03-21', '2011-08-22'],
                          "The Avengers": ['2011-10-11', '2012-06-04']},
                'Phase 2': {"Iron Man 3": [ '2012-10-23', '2013-05-14'], 
                            "Thor: The Dark World": ['2013-04-24', '2013-11-22'],
                            "Captain America: The Winter Soldier": [ '2013-10-25', '2014-04-13'],
                            "Guardians of Galaxy": ['2014-02-18', '2014-09-01'], 
                            "Avengers: Age of Ultron": ['2014-10-23', '2015-06-01'],
                            "Ant-Man": ['2015-04-13', '2015-08-17']},
                'Phase 3': {"Captain America: Civil War": [ '2016-04-12', '2016-06-06'],
                            "Doctor Strange": ['2016-04-13', '2016-12-04'],
                            "Guardians of Galaxy Vol.2": [ '2016-12-04', '2017-06-05'],
                            "Spider-Man: Homecoming": ['2016-12-11', '2017-08-07'],
                            "Thor: Ragnarok": ['2017-04-10', '2017-11-10'],
                            "Black Panther": ['2017-06-10', '2018-03-16'],
                            "Avengers: Infinity War": ['2018-03-16', '2018-05-27'],
                            "Ant-Man and the Wasp": ['2018-01-20', '2018-08-06'],
                            "Captain Marvel": ['2018-09-18', '2019-04-08'],
                            "Avengers: Endgame": ['2019-03-14', '2019-05-26'],
                            " Spider-Man: Far From Home": ['2019-01-15', '2019-08-02']},
                'Phase 4': {"Black Widow": ['2019-12-03','2021-08-09'],
                            "Shang-Chi and the Legend of the Ten Rings": ['2021-4-19', '2021-10-3'],
                            "Eternals":['2021-05-24', '2021-12-05'],
                            "Spider-Man: No Way Home": ['2021-08-23', '2022-01-14'],
                            "Doctor Strange in the Multiverse of Madness": ['2021-12-15', '2022-06-06']}}

In [ ]:
import json
def loop_fII():
  # Per ogni fase e per ogni film
  for phase in film_title:
    for title in film_title[phase]:
      search = '{} Twitter'.format(title)
      start_date_input = datetime.strptime(film_title[phase][title][0], '%Y-%m-%d')
      end_date_input = datetime.strptime(film_title[phase][title][1], '%Y-%m-%d')

      # Costruisci una lista con tutti i periodi mensili tra start_date_input e end_date_input. Obbiettivo: superare il limite di 100 articoli scaricabili 
      a = ((end_date_input-start_date_input)/31).days    # numero totale di mesi
      delta = timedelta(days= 31)
      start_list = [start_date_input + (i*delta) for i in range(a+1)]
      start_list.append(end_date_input)  

      # Costruisci un dizionario i cui elementi sono i dataset ottenuti iterando la funzione per ogni coppia (star/end_date) in start_list 
      try:  
        dictio = {el: get_storyII(search, start_list[el],start_list[el+1]) for el in range(len(start_list)-1)}  
      except:
        print('Notizie non trovate per questo film')
        continue

      # Concatena i dataset in un unico dataset
      Scraping_news = dictio[0]
      for x in range(1,len(dictio)):
        # Evita che concateni dataset vuoti
        if len(dictio[x]) != 0:
          Scraping_news = pd.concat([Scraping_news, dictio[x]], ignore_index=True)  
        else:
          print('Notizie non trovate per un mese')
          continue
      
        # Uniforma l'attributo 'published'
        Scraping_news['published'] = pd.to_datetime(Scraping_news['published']) 
        Scraping_news['published'] = Scraping_news['published'].dt.strftime('%d/%m/%Y')
        
        # Salviamo il dataset in una cartella sul drive 
        # Scraping_news.to_csv(('/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/JSON SCRAPING TOM/{} scraping_news.csv').format(search))
        Scraping_news.to_json(('/content/drive/MyDrive/DATA SCIENCE/Progetti I anno/Dataset/JSON SCRAPING TOM/{} scraping_news.json').format(search), orient='records')

  return('Scraping completato')

In [ ]:
loop_fII()